In [ ]:
# Graph Algorithms with NetworkX and Matplotlib
# Author: Félix Watine
# Date: Jan 2024

import networkx as nx
import matplotlib.pyplot as plt

# 1. Example of adjacency Lists for Graphs
S1 = {0: {1},  # Node 0 is connected to Node 1
      1: {2},  # Node 1 is connected to Node 2
      2: {0},  # Node 2 is connected to Node 0
      3: {4}}  # Node 3 is connected to Node 4

S2 = {0: {1, 3, 4},
      1: {0},
      2: {3},
      3: {0, 2},
      4: {0, 5},
      5: {4, 6},
      6: {5},
      7: {8},
      8: {7}}

# 2. Plotting the Graph
# Uncomment the following lines to visualize the graph

# G = nx.Graph(S1)
# pos = nx.spring_layout(G)  # Choose a layout algorithm (e.g., spring layout)
# nx.draw(G, pos, with_labels=True, node_color='lightblue', node_size=500, edge_color='gray')
# plt.show()

# 3. Breadth-First Search (BFS)
def bfs(Adj, s):
    parent = [None for _ in Adj]  # Initialize parent array
    parent[s] = s  # The parent of the start node is itself
    level = [[s]]  # Initialize the level array
    while level[-1]:  # Continue while the last level is not empty
        level.append([])
        for u in level[-2]:  # Visit all nodes in the last level
            for v in Adj[u]:  # Visit all adjacent nodes
                if parent[v] is None:  # If the node is not yet visited
                    parent[v] = u
                    level[-1].append(v)
    return level, parent

# 4. Unweighted Shortest Path
def usp(Adj, s, t):
    parent = bfs(Adj, s)[1]  # Get the parent array from BFS
    if parent[t] is None:
        return None
    path = [t]
    while path[-1] != s:  # Build the path from target to source
        path.append(parent[path[-1]])
    return path[::-1]  # Return the path in the correct order

# 5. Depth-First Search (DFS)
def dfs(Adj, s, parent=None, order=None):
    if parent is None:
        parent = [None for _ in Adj]  # Initialize parent array
        parent[s] = s  # The parent of the start node is itself
        order = []  # Initialize the order array
    for v in Adj[s]:
        if parent[v] is None:
            parent[v] = s
            dfs(Adj, v, parent, order)
    order.append(s)
    return parent, order

# 6. Full Graph Exploration
def fge(Adj):
    parent = [None for _ in Adj]
    order = []
    for v in Adj:
        if parent[v] is None:
            parent[v] = v
            parent, order = dfs(Adj, v, parent, order)
    return parent, order

# 7. Adjacency Lists for Weighted Graphs
W1 = {0: {1: 1, 3: 3, 4: 2, 7: 2},
      1: {0: 1},
      2: {3: 1},
      3: {0: 3, 2: 1},
      4: {0: 2, 5: 1},
      5: {4: 1, 6: 3},
      6: {5: 3, 7: 1},
      7: {0: 2, 6: 1}}

W2 = {0: {1: 1, 3: 3, 4: 2, 7: 2},
      1: {},
      2: {},
      3: {2: 1},
      4: {0: 2},
      5: {4: 1},
      6: {5: 3},
      7: {6: 1}}

# 8. Plotting Weighted Graph
# Uncomment the following lines to visualize the weighted graph

# G = nx.Graph(W1)
# G.add_weighted_edges_from([(u, v, weight) for u, neighbors in W1.items() for v, weight in neighbors.items()])
# pos = nx.spring_layout(G)
# nx.draw(G, pos, with_labels=True, node_color='lightblue', node_size=500, edge_color='gray')
# edge_labels = nx.get_edge_attributes(G, 'weight')
# nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels)
# plt.show()

# 9. Weighted Single Source Shortest Path Algorithm - DAG Relaxation
def try_to_relax(Adj, d, parent, u, v):
    if d[v] > d[u] + Adj[u][v]:
        d[v] = d[u] + Adj[u][v]
        parent[v] = u
    return d, parent

def DAG_Relaxation(Adj, s):
    _, order = dfs(Adj, s)
    order.reverse()
    d = [float("inf") for _ in Adj]
    parent = [None for _ in Adj]
    d[s] = 0
    parent[s] = s
    for u in order:
        for v in Adj[u]:
            try_to_relax(Adj, d, parent, u, v)
    return d, parent

# 10. Weighted Single Source Shortest Path Algorithm - Bellman-Ford
def bellman_ford(Adj, s):
    d = [float("inf") for _ in Adj]
    parent = [None for _ in Adj]
    d[s], parent[s] = 0, s
    V = len(Adj)
    for _ in range(V - 1):
        for u in range(V):
            for v in Adj[u]:
                try_to_relax(Adj, d, parent, u, v)
    # Check for negative weight cycles
    for u in range(V):
        for v in Adj[u]:
            if d[v] > d[u] + Adj[u][v]:
                raise Exception("Negative weight cycle detected!")
    return d, parent

# 11. Weighted Single Source Shortest Path Algorithm - Dijkstra
class PriorityQueue:
    def __init__(self):
        self.A = {}

    def insert(self, label, key):
        self.A[label] = key

    def extract_min(self):
        label_min = min(self.A, key=self.A.get)
        return label_min

    def remove_min(self):
        label_min = self.extract_min()
        del self.A[label_min]

    def decrease_key(self, label, key):
        self.A[label] = key

def dijkstra(Adj, s):
    d = [float("inf") for _ in Adj]
    d[s] = 0
    parent = [None for _ in Adj]
    parent[s] = s
    pq = PriorityQueue()
    for v in Adj:
        pq.insert(v, d[v])
    while pq.A:
        v = pq.extract_min()
        for u in Adj[v]:
            if d[u] > d[v] + Adj[v][u]:
                d[u] = d[v] + Adj[v][u]
                parent[u] = v
                pq.decrease_key(u, d[u])
        pq.remove_min()
    return d, parent
